### Import
---



In [6]:
import numpy as np
import random as rand
from note_include.elem.Ring   import Ring
from note_include.elem.LWE    import LWE
from note_include.elem.RLWE   import RLWE
from note_include.elem.RLWEp  import RLWEp
from note_include.elem.RGSW   import RGSW
from note_include.FHEW        import FHEW
from note_include.utils.types import RGSWctxt, RLWEctxt, RLWEpctxt, LWEctxt

np.set_printoptions(threshold=np.inf, linewidth=np.inf)


---

# CGGI

* <a href="https://eprint.iacr.org/2016/870"> Faster Fully Homomorphic Encryption: Bootstrapping in less than 0.1 Seconds </a>

* <a href="https://eprint.iacr.org/2020/086"> Bootstrapping in FHEW-like Cryptosystems </a>

이전 장에서는 DM-like blind rotation 과 FHEW 에 중요한 함수인 ACC initialization 과 LWE extraction 에 대해서 알아보았다.

이번에는 bianry, ternary key 에서 최강 성능을 자랑하는 CGGI 에 대해서 알아 볼 것이다.

DM 과 차이를 보이는 blind rotation 에 대해서만 중점적으로 다룰 예정이므로 DM 을 먼저 보고 오는 것을 추천한다.

---

Blind rotation 을 수행하는 이유는 $X^t$ 에서 $t = \langle \vec{a}, \vec{s} \rangle$ 일 때, $X^t$ 의 계수를 상수항으로 안전하게 끌고오기 위함이었음을 기억하자.

간단하게 생각해서 $X^{-t}$ 를 accumulator, $\textsf{acc}$ 에 곱해주는 동작이라고 생각할 수 있다.

DM 에서는 이를 수행하기 위해 key switching 에서 사용했던 테크닉과 유사하게 동작한다.

CGGI 는 이를 약간 변형한 것이다. 우선 blind rotation key 가 어떤식으로 정의되는지 확인해보자.

>$\textbf{Definition : CGGI-like Blind rotation key}$
>
>비밀키 $\vec{s}$ 가 있을 때 CGGI-like blind rotation key, $\textsf{brk}$ 는 다음과 같이 정의된다:
>
>\begin{align*}
>    \textsf{brk}_{iu} = \left\{
>        \text{RGSW}(x_{iu}) | \vec{x}_i \in \{0,1\}^{|U|} \text{ s.t. } \sum_{u \in U} u \cdot x_{iu} = s_i
>    \right\}, \text{ where } U \subset \mathbb{Z}_q.
>\end{align*}

수식만 보고는 왜 이런식으로 blind rotation key 를 정의하는지 잘 와닿지 않을 수 있다.

쉽게 말해서 $a_i \in \vec{a}$ 를 분해하여 $\textsf{brk}$ 를 만들었던 DM 과 달리 CGGI 는 $s_i \in \vec{s}$ 를 분해하여 $\textsf{brk}$ 를 만든다는 것이다.

수식으로 표현해보자. DM 같은 경우에는 마치 다음과 같이 $a_i \cdot s_i$ 를 수행했다.

\begin{align*}
    a_i \cdot s_i = \sum_{j=0}^{d-1} \hat{a}_j \cdot s_i \cdot B^j, \text{ where } (\hat{a})_j = \textsf{gadget\_decomposition\_int}(a_i).
\end{align*}

반대로 CGGI 는 다음과 같이 표현할 수 있을 것이다.

\begin{align*}
    a_i \cdot s_i = \sum_{u \in U} a_i \cdot x_{iu}, \text{ where } x_{iu} \in \{0,1\} \text{ and } \sum_{u \in U} u \cdot x_{iu} = s_i.
\end{align*}

이해를 돕기 위해 약간의 예시를 들어보자. Secret key distribution 이 binary 인 경우 즉, $s_i \in \{0,1\}$ 인 경우에 $U$ 는 무엇일까?

이 경우 $U = \{1\}$ 로 설정할 수 있다.

* $s_i = 0$ 인 경우, $1 \cdot x_{iu} = 0, \text{ where } x_{iu} = 0$
* $s_i = 1$ 인 경우, $1 \cdot x_{iu} = 1, \text{ where } x_{iu} = 1$

그럼 ternary 즉, $s_i \in \{-1,0,1\}$ 인 경우를 고려해보자.

이 경우에는 U = \{-1,1\} 로 설정할 수 있다.

* $s_i = 0$ 인 경우, $1 \cdot x_{iu1} + 1 \cdot x_{iu2} = 0, \text{ where } x_{iu1} = 0, x_{iu2} = 0$
* $s_i = 1$ 인 경우, $1 \cdot x_{iu1} + 1 \cdot x_{iu2} = 1, \text{ where } x_{iu1} = 1, x_{iu2} = 0$
* $s_i = -1$ 인 경우, $-1 \cdot x_{iu1} + 1 \cdot x_{iu2} = -1, \text{ where } x_{iu1} = 1, x_{iu2} = 0$

이 경우 key size 가 $|U|$ 에 따라 늘어난다는 사실을 금방 알 수 있다.

그럼 gaussian 즉, $s_i \in \mathbb{Z}_q$ 인 경우엔 어떨까?

이 경우에 $U$ 는 $U = \{2^0, 2^1, 2^2, \dots, 2^{\lceil\log_2q\rceil - 1} \}$ 로 표기되어 key size 가 엄청 늘어나게 된다.

따라서 CGGI 는 binary 혹은 ternary 에서 매우 우수한 성능을 보이는데 반해 gaussian 에서는 효율적이지 않다.

마지막으로 Blind rotation 자체는 어떤식으로 정의되는지 확인해보자.

>$\textbf{Definition : CGGI-like Blind rotation}$
>
>Accumulator $\textsf{acc}$ 와 blind rotation key $\textsf{brk}_{iu}$ 가 주어졌을 때, CGGI-like blind rotation 은 다음과 같이 정의된다:
>
>\begin{align*}
>    \textsf{Blind\_Rotation}(\textsf{acc}, \textsf{brk}) : 
>    \textsf{acc} \leftarrow \textsf{acc} + (X^{a_i} - 1) \cdot (\textsf{acc} \odot' \textsf{brk}_{iu}) \text{ for all } u \in U. 
>\end{align*}

이러한 동작이 마치 $x_{iu}$ 가 1일 때는 선택하고 $x_{iu}$ 가 0일 때는 무시하는 것으로 보이기에 Homomorphic CMUX bootstrapping 이라고 불리기도 한다.

코드를 통해 확인해보자!

In [7]:
# def BRKgen(self, s:list[int], s_ring:Ring) -> list[RGSWctxt]: # RGSW(X^{-s_i}) for i \in [0, n)
#     brk = []
#     for _s in s:
#         monomial = np.zeros(self.N)
#         monomial[0] = _s
#         monomial = Ring(self.N, self.Q, monomial)
#         brk.append(self.RGSW_CC.encrypt(monomial, s_ring))
#     return brk

# def Blindrotation(self, ctxt_operand:LWEctxt, acc:RLWEctxt, brk:list[RGSWctxt]) -> RLWEctxt:
#     a,_ = ctxt_operand
#     N   = int(self.N)        
#
#     # Blind rotation
#     for i, _a in enumerate(a):
#         if(_a == 0):continue
#         monomial = np.zeros(self.N)
#
#         reduced = False
#         if _a >= self.N:
#             _a -= self.N
#             reduced = True
#
#         if _a == 0 and reduced == True:
#             monomial[0]    = -2
#         elif _a == 0 and reduced == False:
#             continue
#         elif reduced == True:           
#             monomial[N-_a] =  1
#             monomial[0]    = -1            
#         elif reduced == False:          
#             monomial[N-_a] = -1
#             monomial[0]    = -1
#
#         monomial     = Ring(self.N, self.Q, monomial)
#         tmp = acc
#         acc = self.RGSW_CC.mult_rlwe(acc, brk[i])
#         acc = self.RLWE_CC.mult_ring_ptxt(acc, monomial)
#         acc = self.RLWE_CC.add_ctxt_ctxt(tmp, acc)
#
#     return acc

In [8]:
q      = 128
n      = 16
Q      = 128
N      = 64
B      = 32
B_ks   = 32
s_std  = 'Binary'
e_std  = 0.

fhew = FHEW(n, q, N, Q, s_std, e_std, B, B_ks, method='CGGI')
s, s_ring = fhew.keygen()

m0  = 0
m1  = 1

ct00 = fhew.encrypt(m0, s)
ct01 = fhew.encrypt(m0, s)
ct10 = fhew.encrypt(m1, s)
ct11 = fhew.encrypt(m1, s)

ct_and_00 = fhew.evalBin(ct00, ct01, "AND")
ct_and_01 = fhew.evalBin(ct00, ct11, "AND")
ct_and_10 = fhew.evalBin(ct10, ct01, "AND")
ct_and_11 = fhew.evalBin(ct10, ct11, "AND")

print("0 AND 0 = ", fhew.decrypt(ct_and_00, s))
print("0 AND 1 = ", fhew.decrypt(ct_and_01, s))
print("1 AND 0 = ", fhew.decrypt(ct_and_10, s))
print("1 AND 1 = ", fhew.decrypt(ct_and_11, s))

0 AND 0 =  0
0 AND 1 =  0
1 AND 0 =  0
1 AND 1 =  1



---

## Summary

이번 장에서는 CGGI 에 대해서 알아보았다.

DM 과는 달리 binary, ternary 에 강한 모습을 보인다.

---

## Code implementation

`Note/note_include/FHEW.py` 에서 ACC init, LWE extract 등을 확인할 수 있고,

`Note/note_include/blindrotations/CGGI.py` 에서 CGGI-like blind rotation 관련 코드들을 확인할 수 있다.